# **BILBIOTHEQUES**

In [1]:
import json

import pandas as pd
import numpy as np

import chess
import random

from Fct.fct_preprocess import *
from Fct.fct_eval import *

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D, Flatten, Dropout, Dense, BatchNormalization, MaxPooling2D
from tensorflow.keras.optimizers import Adam


import warnings
warnings.filterwarnings("ignore")

---

# **1. EXTRACTION DES DONNEES BRUTES**

Extraction des données depuis le fichier JSONL d'origine qui contient 137 000 000 positions différentes, chacune avec plusieurs évaluations stockfish . Nous limitons alors la taille de notre base de donnée a **500 000 positions**.

In [2]:
file_path = 'data\lichess_db_eval.jsonl'
data_raw = extract_data(file_path, max_rows=1000000)
data_raw.head(5)

,fen,knodes,depth,cp,mate,line
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2
3,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,64.0,NaN,f7g7 e6e2 g7g6 d1c2 h8d8 e2d2 g6f6 a2a3 b7b5 c4a2
4,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,59730,31,134.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4b3 a5a4 b3a2 c6d7 e6h6


La base de donnée est composée de **2 330 098 évaluations différentes** et **5 variables** : 

| Variable            | Description                                                                                                        |
|-----------------|--------------------------------------------------------------------------------------------------------------------|
| **knodes**      | Nombre de milliers de nœuds analysés par stockfish pendant la recherche de la position.                   |
| **depth**       | Profondeur de la recherche, indiquant le nombre de coups (ou niveaux) explorés par le moteur.                     |
| **cp**          | Évaluation en centipions, représentant un avantage matériel en fonction du côté actif (positif pour Blancs, négatif pour Noirs). |
| **mate**        | Évaluation de mat. Si une valeur est donnée, elle indique le nombre de coups restants avant que le mat soit atteint. |
| **line**        | Ligne (parfois appelée variante) désigne une séquence de coups qui découle d'une position donnée. Elle représente une possible continuation du jeu |

---

# **2. NETTOYAGE DES DONNEES**

Stockfish utilie différents parametre dans son algorythme, notamment le **nombre de noeuf (knodes)** et le la **profondeur (depth)**. En fonction de ces différents parametres, stockfish calcul le **centipions (cp)** et **l'évaluation du mate (mate)**.  

Après ces évaluations, stockfish peut alors parfois prédire des coups différents pour une même position, on appelle cela des **variantes**. Ces coups sont différents mais ont un impacte similaire ou quasi-similaire sur l'avantage qu'il donne a celui qui le joue (en terme d'evaluation de la position). 

Certains joueurs vont être plus a l'aise dans une variante plûtot qu'une autre en fonction de leurs styles de jeu ou de leurs connaissances, mais pour une IA cela n'as pas d'importance ! Toutefois, notre source de donnée nous propose plusieurs analyse de stockfish pour chaques position, apportant son lot de variante.

Nous pouvons alors utiliser 2 méthodes pour garder la meilleur variante.

### **Méthode N°1 : Analyse des Parametres et de l'Evaluation**
   1. **Profondeur de recherche (depth)** : Plus la profondeur est grande, plus l'analyse est précise. Priorisations des évaluations avec la profondeur maximale.

   2. **Centipions (cp) ou mat (mate)** : Si une évaluation donne un **mate**, elle est prioritaire, car un mat forcé est absolu. En absence de **mate**, choix d'évaluation avec la valeur **cp** la plus élevée pour le joueur actif

   3. **Nombre de nœuds analysés (knodes)** : Si plusieurs évaluations ont la même profondeur, celle ayant exploré le plus grand nombre de nœuds est théoriquement plus fiable.

   4. **En cas d'égalité parfaite**: En cas d’égalité sur les autres critères, utilisez une évaluation arbitraire.

### **Méthode N°2 : Coup le Plus Populaire** 
blablablabla


- Création d'une variable `best_move_m1`, qui extrait le prochain coup a jouer, c'est a dire le premier coup de la ligne.

In [3]:
data_V1 = extract_next_move(data_raw)
data_V1.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8


- Création d'une variable `best_move_m2` avec le coup le plus populaire par position en suivant la Méthode N°2

In [4]:
data_V2 = most_popular_predict_V1(data_V1)
data_V2.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,200973,39,58.0,NaN,f7g7 e6e2 h8d8 e2d2 b7b5 c4e6 g7f6 e6b3 a6a5 a2a3,f7g7,f7g7
1,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,62.0,NaN,f7g7 e6e2 b7b5 c4b3 h8d8 e2d2 a6a5 a2a3 g7f6 d1e1,f7g7,f7g7
2,7r/1p3k2/p1bPR3/5p2/2B2P1p/8/PP4P1/3K4 b - -,71927,32,151.0,NaN,h8d8 d1e1 a6a5 a2a3 b7b5 c4a2 c6d7 e6e7 f7g6 e1f2,h8d8,f7g7


- Filtrage des lignes pour ne garder qu'une evaluation par position selon la méthode N°1.

In [5]:
data_V3 = filter_best_predict(data_V2)
data_V3.head(3)

,fen,knodes,depth,cp,mate,line,best_move_m1,best_move_m2
0,1B1R4/4r1bk/1pb1B1pp/2p5/8/P1P3P1/5P1P/6K1 b - -,9028,21,0.0,NaN,c6e8 e6d5 g7c3 b8d6 e7e1 g1g2 e8b5 d6c7 c3d4 g3g4,c6e8,e7e6
1,1B1b4/2p2ppp/Bp2b3/3k4/1P6/P7/2K2PPP/8 b - -,13863,23,-132.0,NaN,d5c6 b8a7 e6f5 c2b3 d8f6 h2h3 f6d4 f2f3 d4e5 a3a4,d5c6,e6d7
2,1B1b4/2p2ppp/Bp2b3/8/1P2k3/P7/2K2PPP/8 w - -,57975,28,-4.0,NaN,a3a4 e4d5 a6b5 c7c6 b5e2 c6c5 b4c5 d5c5 b8e5 f7f6,a3a4,a3a4


- Suppression des variables inutiles. Les variables `knodes`, `depth`, `cp`, `mate` et `line` ne sont plus utile pour la suite du projet

In [6]:
data_cleaned = drop_usuless_columns(data_V3)
data_cleaned.head(5)

,fen,best_move_m1,best_move_m2
0,1B1R4/4r1bk/1pb1B1pp/2p5/8/P1P3P1/5P1P/6K1 b - -,c6e8,e7e6
1,1B1b4/2p2ppp/Bp2b3/3k4/1P6/P7/2K2PPP/8 b - -,d5c6,e6d7
2,1B1b4/2p2ppp/Bp2b3/8/1P2k3/P7/2K2PPP/8 w - -,a3a4,a3a4
3,1B1b4/8/2pP2p1/5p2/k7/8/5PK1/8 b - -,d8b6,d8b6
4,1B1b4/8/8/8/4k1P1/6K1/8/8 w - -,g3h3,g3h3


- Sauvegarde des données netoyées en CSV

In [7]:
data_cleaned.to_csv('data/data_cleaned.csv')

---

# **3. ENTRAINEMENT DES MODELES**

blablablabla

- Chargement des Données

In [2]:
data = pd.read_csv('data/data_cleaned.csv' )

X = data['fen']
y = data['best_move_m1']

- Traduction des données sous une forme interpretable par le reseau de neurone.
  - **Position (Fen)** -> Matrice Numpy
  - **Coup a joué (UCI)** -> Encodage de chaque coup distincts (sous forme d'entier)

In [3]:
X = X.apply(fen_to_matrix)
X = np.array(X.tolist())

# Encode les mouvements et conversion en catégories
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))


# Sauvegarde de move_to_int dans un fichier JSON
with open('Models/move_int_dico.json', 'w') as file:
    json.dump(move_to_int, file)


KeyboardInterrupt



### **Modèle N°1** :


   - **`Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12))`** : 
      - C'est une couche de convolution 2D qui extrait des caractéristiques spatiales de l'échiquier. 
      - **64 filtres** sont utilisés pour capter des motifs complexes, avec une taille de filtre de **3x3**, idéale pour des images petites comme un échiquier **8x8**. 
      - La fonction d'activation **ReLU** accélère l'apprentissage en gérant les gradients de manière efficace.

- **`Conv2D(128, (3, 3), activation='relu')`** : 
   - Une deuxième couche de convolution avec **128 filtres** permet de capturer des motifs plus complexes et abstraits sur l'échiquier.
   - La taille des filtres reste de **3x3**, et **ReLU** est utilisé pour améliorer l'apprentissage.

- **`Flatten()`** : 
   - Cette couche aplatit les sorties **2D** des couches précédentes en un vecteur **1D**, ce qui est nécessaire pour connecter les couches de convolution aux couches entièrement connectées.

- **`Dense(256, activation='relu')`** : 
   - Une couche entièrement connectée avec **256 neurones** pour capturer des relations non-linéaires complexes entre les caractéristiques extraites par les convolutions.
   - La fonction **ReLU** est utilisée pour une meilleure gestion des non-linéarités.

- **`Dense(len(move_to_int), activation='softmax')`** : 
   - La couche de sortie a un nombre de neurones égal au nombre de **coups possibles**. La fonction **Softmax** est utilisée pour classer les coups possibles en leur attribuant une probabilité, ce qui permet de sélectionner le coup le plus probable.

- **Définition du modèle**

In [6]:
model_1 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])

model_1.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model_1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 9, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,179,904 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1454)           │       373,678 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,632,110 (6.23 MB)

 Trainable params: 1,632,110 (6.23 MB)

 Non-trainable params: 0 (0.00 B)

- **Entrainement du modèle**

blablablalba

In [7]:
model_1.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

Epoch 1/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 29ms/step - accuracy: 0.0084 - loss: 6.9599 - val_accuracy: 0.0370 - val_loss: 4.9291
Epoch 2/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.0281 - loss: 6.2305 - val_accuracy: 0.0700 - val_loss: 4.4015
Epoch 3/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.0373 - loss: 5.7623 - val_accuracy: 0.0960 - val_loss: 4.2521
Epoch 4/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.0722 - loss: 5.2460 - val_accuracy: 0.1220 - val_loss: 4.1517
Epoch 5/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.1181 - loss: 4.5947 - val_accuracy: 0.0830 - val_loss: 4.3362
Epoch 6/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.1884 - loss: 3.8623 - val_accuracy: 0.1250 - val_loss: 4.3978
Epoch 7/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.3090 - loss: 2.9951 - val_accuracy: 0.1210 - val_loss: 4.4374
Epoch 8/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.4214 - loss: 2.2343 - val_accu

- **Sauvegarde du Modèle**

In [8]:
model_1.save("Models/Modele_1_TF_25EPOCHS.keras")

### **Modèle N°2** :

Ajout de plus de couches de convolution permet de capturer des motifs encore plus complexes. Cependant, cela peut augmenter le risque de sur-apprentissage, donc il faudra ajuster les autres hyperparamètres (comme le taux d'apprentissage ou la régularisation).

- **Définition du modèle**

In [22]:
model_2 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    Conv2D(128, (3, 3), activation='relu'),
    Conv2D(256, (3, 3), activation='relu'),  # Ajout d'une couche supplémentaire
    Flatten(),
    Dense(512, activation='relu'),  # Augmentation de la taille de la couche dense
    Dense(len(move_to_int), activation='softmax')
])

model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model_2.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_11 (Conv2D)              │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 9, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 7, 2, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 3584)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │     1,835,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1454)           │       745,902 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,955,118 (11.27 MB)

 Trainable params: 2,955,118 (11.27 MB)

 Non-trainable params: 0 (0.00 B)

- **Entrainement du modèle**

In [23]:
model_2.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

Epoch 1/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 28ms/step - accuracy: 0.0091 - loss: 6.9579 - val_accuracy: 0.0160 - val_loss: 4.9129
Epoch 2/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - accuracy: 0.0243 - loss: 6.2549 - val_accuracy: 0.0770 - val_loss: 4.4433
Epoch 3/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 38ms/step - accuracy: 0.0290 - loss: 5.8038 - val_accuracy: 0.0740 - val_loss: 4.3501
Epoch 4/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 43ms/step - accuracy: 0.0500 - loss: 5.3645 - val_accuracy: 0.1160 - val_loss: 4.2339
Epoch 5/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 44ms/step - accuracy: 0.0824 - loss: 4.8405 - val_accuracy: 0.1190 - val_loss: 4.3054
Epoch 6/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 47ms/step - accuracy: 0.1494 - loss: 4.1353 - val_accuracy: 0.1230 - val_loss: 4.2991
Epoch 7/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.2346 - loss: 3.3744 - val_accuracy: 0.1240 - val_loss: 4.3689
Epoch 8/25
141/141 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.3541 - loss: 2.5238 - val_accu

- **Sauvegarde du Modèle**

In [24]:
model_2.save("Models/Modele_2_TF_25EPOCHS.keras")

### **Modèle N°3** :

Moins de neurones dans la couche dense peut rendre le modèle plus rapide à entraîner et plus facile à régulariser, tout en conservant une bonne capacité de généralisation.

- Definition du Modèle

In [15]:
model_3 = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(13, 8, 8)),
    BatchNormalization(),  # Ajout de BatchNormalization pour stabiliser l'entraînement
    MaxPooling2D(pool_size=(2, 2)),  # Réduction de la dimension spatiale
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dropout(0.4),  # Dropout réduit à 40% ici pour limiter le surapprentissage
    Dense(256, activation='relu'),
    Dropout(0.3),  # Dropout progressif pour éviter un réseau trop complexe
    Dense(len(move_to_int), activation='softmax')  # Couche de sortie
])

# Optimiseur avec un learning rate ajusté
model_3.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

model_3.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 11, 6, 64)      │         4,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 11, 6, 64)      │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 5, 3, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 3, 1, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 3, 1, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 1, 0, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 0)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1927)           │       495,239 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 574,791 (2.19 MB)

 Trainable params: 574,407 (2.19 MB)

 Non-trainable params: 384 (1.50 KB)

- Entrainement du Modèle

In [ ]:
model_3.fit(X, y, epochs=25, validation_split=0.1, batch_size=64)

- Sauvegarde du modèle

In [27]:
model_3.save("Models/Modele_3_TF_25EPOCHS.keras")

# **4. EVALUATION DES MODELES**

In [2]:
# Chargement des différents models

model_1 = load_model("Models\Modele_1_TF_25EPOCHS.keras")
model_2 = load_model("Models\Modele_2_TF_25EPOCHS.keras")
model_3 = load_model("Models\Modele_3_TF_25EPOCHS.keras")

# Chargement du dictionnaire de traduction coup / integer pour l'interpretation des modèles
with open('Models/move_int_dico.json', 'r') as file:
    move_int = json.load(file)
move_int_dico = {v: k for k, v in move_int.items()}

In [3]:
play_game(model_1,"random",move_int_dico,print_game = True)

Début de la partie !
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . P . . . .
P P P . P P P P
R N B Q K B N R 

r n b q k b n r
p p p p p p p .
. . . . . . . .
. . . . . . . p
. . . . . . . .
. . . P . . . .
P P P . P P P P
R N B Q K B N R 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
r n b q k b n r
p p p p p p p .
. . . . . . . .
. . . . . . . p
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R 

r n b q k b n r
p p p . p p p .
. . . p . . . .
. . . . . . . p
. . . P . . . .
. . . . . . . .
P P P . P P P P
R N B Q K B N R 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
r n b q k b n r
p p p . p p p .
. . . p . . . .
. . . . . . . p
. . . P . . . P
. . . . . . . .
P P P . P P P .
R N B Q K B N R 

r n b q k b n r
p p p . p . p .
. . . p . . . .
. . . . . p . p
. . . P . . . P
. . . 

'1-0'

A FAIRE -> METTRE EN PLACE LA NUL PAR REPETITION OU A 100 COUPS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# fonction cassé a réparé :

def play_multiple_games(ia1, ia2, n, model1=None, model2=None, move_int_dico=None):
    """
    Fait jouer plusieurs parties entre deux IA et retourne les résultats cumulés.

    Arguments:
    - ia1, ia2 : Fonctions des IA pour les joueurs.
    - n : Nombre de parties à jouer.
    - model1, model2 : Modèles TensorFlow (facultatifs).
    - move_int_dico : Dictionnaire pour les prédictions (facultatif).

    Retourne:
    - Résumé des résultats : victoires IA1, victoires IA2, égalités.
    """
    ia1_wins = 0
    ia2_wins = 0
    draws = 0

    for i in range(n):
        print(f"\nPartie {i+1}:")
        moves, result = play_game_auto(ia1, ia2, model1, model2, move_int_dico)
        
        if result == "1-0":
            ia1_wins += 1
        elif result == "0-1":
            ia2_wins += 1
        else:
            draws += 1

    # Résultats
    print("\nRésultats finaux:")
    print(f"Victoires IA1 : {ia1_wins}")
    print(f"Victoires IA2 : {ia2_wins}")
    print(f"Égalités : {draws}")

    return {"IA1 Wins": ia1_wins, "IA2 Wins": ia2_wins, "Draws": draws}
